In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import math

In [2]:
data=pd.read_csv('~\Downloads\daily_data.csv')
data

,datetime,high,low,close,open,volume
0,2018-01-01,13818.55,12750.00,13135.00,13715.65,6970.441076
1,2018-01-02,14750.00,12890.02,14580.00,13135.00,16359.347340
2,2018-01-03,15473.49,14150.00,14749.97,14576.87,19457.238601
3,2018-01-04,15280.00,13918.04,14811.00,14710.00,18783.876561
4,2018-01-05,17118.13,14600.00,16579.00,14805.01,22383.657333
...,...,...,...,...,...,...
1468,2022-01-08,42246.37,40501.00,40679.82,41707.04,27031.900870
1469,2022-01-09,42300.00,40520.00,41966.99,40679.82,27291.009660
1470,2022-01-10,42786.70,39650.00,41604.58,41966.99,51064.904360
1471,2022-01-11,43100.00,41158.99,42755.00,41604.58,38062.818280


In [3]:
def get_macd(price,slow,fast,smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1-exp2).rename(columns = {'close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    frames =  [macd, signal]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

macd=get_macd(data.close,26,12,9)
macd

,macd,signal
0,0.000000,0.000000
1,115.270655,23.054131
2,217.827657,62.008836
3,300.564708,109.720011
4,502.998982,188.375805
...,...,...
1468,-2136.672368,-1568.284564
1469,-2233.642180,-1701.356087
1470,-2313.071268,-1823.699123
1471,-2257.170852,-1910.393469


In [4]:
def generate_signal_for_macd(df,data):
    datetime=[]
    signal=[]
    current=0
    datetime.append(data.datetime[0])
    signal.append(0)
    for i in range(1,len(df.macd)):
        if(df.macd[i]>=df.signal[i] ):
            datetime.append(data.datetime[i])
            signal.append(1)
        elif(df.macd[i]<=df.signal[i]):
            datetime.append(data.datetime[i])
            signal.append(-1)
        else:
            datetime.append(data.datetime[i])
            signal.append(0)
    a=pd.DataFrame(columns=['datetime','signal'])
    a['datetime']=datetime
    a['signal']=signal
    return a
            
        
        
output=generate_signal_for_macd(macd,data)
output

,datetime,signal
0,2018-01-01,0
1,2018-01-02,1
2,2018-01-03,1
3,2018-01-04,1
4,2018-01-05,1
...,...,...
1468,2022-01-08,-1
1469,2022-01-09,-1
1470,2022-01-10,-1
1471,2022-01-11,-1


In [5]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signal[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signal[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime'])
    to_submit['datetime']=datetime
    to_submit['open']=data.open
    
    to_submit['high']=data.high
    to_submit['low']=data.low
    to_submit['close']=data.close
    to_submit['volume']=data.volume
    
    to_submit['close\open']=trade
#     to_submit['price']=price
    to_submit['signal']=signal

    return to_submit
            
            

In [116]:
def trailing_stop_loss(data,submit):
    datetime=[]
    trade=[]
    signal=[]
    percent=0.05
    hello=0
    stop_loss=0
    current=0
    l=-1
    for i in range(len(submit.datetime)):
        datetime.append(submit.datetime[i])
        trade.append('cloase')
        if(i<=l):
            continue
        if(hello==0):
            if(submit.signal[i]==0):
                signal.append(0)
            elif(submit.signal[i]==1):
                hello=1
                current=data.close[i]
                stop_loss=((1-percent)*current)
                signal.append(1)
                
            else:
                hello=-1
                current=data.close[i]
                stop_loss=((1+percent)*current)
                signal.append(-1)          
                
        elif(hello==1):
            if(submit.signal[i]==-1):

                    hello=0
                    signal.append(-1)
                    stop_loss=0
                    
            elif(submit.signal[i]==0):
                if(data.close[i]>current):
                    current=data.close[i]
                    stop_loss=((1-percent)*current)
                    signal.append(0)
            
                    
                elif(data.close[i]<=stop_loss):
                    signal.append(-1)
                    stop_loss=0
                    hello=0
                    for j in range(i+1,len(submit.datetime)):
                        if(submit.signal[j]==-1):
                            signal.append(0)
                            l=j
                            break
                        signal.append(0)
                
                else:
                    signal.append(0)
                        
            
            
        else:#hello=-1
            if(submit.signal[i]==1):
                    signal.append(1)
                    stop_loss=0
                    hello=0
                    
            elif(submit.signal[i]==0):
                if(data.close[i]<current):
                    current=data.close[i]
                    stop_loss=((1+percent)*data.close[i])
                    signal.append(0)
                
                    
                elif(data.close[i]>=stop_loss):
                    signal.append(1)
                    stop_loss=0
                    hello=0
                    for j in range(i+1,len(submit.datetime)):
                        if(submit.signal[j]==1):
                            signal.append(0)
                            l=j
                            break
                        signal.append(0)   
                        
                else:
                    signal.append(0)
                
                
            
        
#     return signal   
    to_submit=pd.DataFrame(columns=['datetime'])
    to_submit['datetime']=datetime
    to_submit['open']=data.open
    
    to_submit['high']=data.high
    to_submit['low']=data.low
    to_submit['close']=data.close
    to_submit['volume']=data.volume
    
    to_submit['close\open']=trade
#     to_submit['price']=price
    to_submit['signal']=signal

    return to_submit 
            
            
    

In [125]:

def only_long_then_short(data,output):
    datetime=[]
    signal=[]
    trade=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        trade.append('close')
#         price.append(data.close[i])
        if(i==len(data.close)-1):
            if(current==1):
                signal.append(-1)
                break
            else:
                signal.append(0)
                break
            
        
        if(output.signal[i]==1):
            if(current==0):
                signal.append(1)
                current=1
            else:
                signal.append(0)
        elif(output.signal[i]==-1):
            if(current==1):
                signal.append(-1)
                current=0
            else:
                signal.append(0)
            
        else:
            if(current==0):
                signal.append(0)
            else:
                signal.append(0)
    a=pd.DataFrame(columns=['datetime'])
    a['datetime']=datetime
    
    a['open']=data.open
    a['high']=data.high
    a['low']=data.low
    a['close']=data.close
    a['volume']=data.volume
    a['close\open']=trade
#     to_submit['price']=price
    a['signal']=signal
    
    
    return a
            

In [150]:
#static approach
def dd_long_short(trade_type,close):
    
    if trade_type == 'Long then short':
        cumulative_max_values = close.cummax()
        cumulative_returns = close/ cumulative_max_values - 1
        max_drawdown = cumulative_returns.min()
        return max_drawdown
        
    elif trade_type == 'Short then long':
        pip=[]
        for i in close:
            pip.append(i)
        stock = -(1000 /pip[0])
        portfolio = []

        for i in range(len(pip)):
            profit = stock * (pip[i] - pip[0])
            portfolio.append(1000+profit)
        
        drawdown = []
        new = 0

        for i in range(len(portfolio)):
            new = max(new,portfolio[i])
            drawdown.append(((new-portfolio[i])/new))
            
        return np.max(drawdown)




def metrics(data,output):
    PL=[]
    PL_in_percent=[]
    hello=0
    number_of_stock=[]
    capital=1000
    entry=[]
    exit=[]
    exit_index=[]
    entry_index=[]
    holding_duration=[]
    drawdown=[]
    dip=[]
    
    commision_rate=1
    risk_free_rate=0.05
    
    for i in range(len(data.close)):
        if(output.signal[i]==1):
            if(hello==0):
                hello=1
                entry_index.append(i)
                entry.append(data.datetime[i])
                
                number_of_stock=(capital)/(data.close[i])
                capital=0
            else:
                hello=0
                exit_index.append(i)
                exit.append(data.datetime[i])
                holding_duration.append(exit_index[-1]-entry_index[-1])
                capital=(capital-(number_of_stock)*(data.close[i]))
                x=entry_index[-1]
#                 if x in data.close.index and i+1 in data.close.index:
                drawdown.append(dd_long_short('Short then long',data.close[x:i+1]))
                dip.append((np.max(data.close[x:i+1])*number_of_stock-1000)/10)    
    
                number_of_stock=0
                PL.append(capital-1000)
                PL_in_percent.append((capital-1000)/10)
                capital=1000
                
        elif(output.signal[i]==-1):
            if(hello==0):
                hello=-1
                
                entry_index.append(i)
                entry.append(data.datetime[i])
                number_of_stock=(capital/(data.close[i]))
                capital=capital+(number_of_stock)*(data.close[i])
            else:
                hello=0
                exit_index.append(i)
                exit.append(data.datetime[i])
                holding_duration.append(exit_index[-1]-entry_index[-1])
                capital=capital+(number_of_stock)*(data.close[i])
                x=entry_index[-1]
                drawdown.append(dd_long_short('Long then short',(data.close[x:i+1])))
                dip.append((1000-(np.min(data.close[x:i+1])*number_of_stock))/10)    
                number_of_stock=0
                PL.append(capital-1000)
                PL_in_percent.append((capital-1000)/10)
                capital=1000
                
                
        
    index=[]            
    for i in range(len(PL)):
        index.append(i)
    a=pd.DataFrame(columns=['trade number','entry','exit','holding duration','return','return %'])
    
    a['trade number']=index
    a['entry']=entry
    a['exit']=exit
    a['holding duration']=holding_duration
    a['return']=PL
    a['return %']=PL_in_percent
    a['drawdown']=drawdown
    a['dip']=dip           
    profit=[]
    loss=[]
    for i in range(len(PL)):
        if(PL[i]>0):
            profit.append(PL[i])
        else:
            loss.append(PL[i])
    print("Total Closed trades: ",len(PL))
    print("Winning trades: ",(len(profit)))
    print("Losing Trades: ",(len(PL)-len(profit)))
    number_of_stock=(1000/(data.close[0]))
    bench_mark_return=(number_of_stock)*(data.close[len(data.close)-1])
    bench_mark_return=(bench_mark_return-1000)
    print("Benchmark Return: ",bench_mark_return)
    print("WIN rate: ",(100*len(profit))/len(PL) , " %")
    print("Gross Profit: ",np.sum(PL))
    net_profit=(np.sum(PL)-(len(PL)*(commision_rate)))
    print("Net Profit: ",net_profit)
    print("Average Profit: ",net_profit/(len(PL)))
    print("MAX Drawdowns: ", (100*(np.min(drawdown)))," %")
    print("Largest Win: ", np.max(profit))
    print("Average Win: ",(np.mean(profit)))
    
   
    print("Largest Loss: ",np.min(loss))
    print("Average Loss: ",np.mean(loss))
    print("Maximum Holding Time: ",np.max(holding_duration))
    print("Average Holding Duration: ",((np.sum(holding_duration))/(len(holding_duration))))
    print("Maximum Dip: ",np.max(dip))
    print("Avg. Dip: ",np.mean(dip))
    print("Sharpe Ratio: ",((np.mean(PL)-risk_free_rate)/(np.std(PL))))
    average_return = np.mean(PL)
    downside_returns = [ret for ret in PL if ret < risk_free_rate]
    downside_volatility = np.std(downside_returns)
    
    sortino_ratio = (average_return - risk_free_rate) / downside_volatility
    print("Sortino Ratio: ", sortino_ratio )
    
   
    print("returns: ",((np.sum(PL)/(10*len(PL))))," %")
    
    return a         
            
    

In [151]:
a=metrics(data,to_submit(data,output))

Total Closed trades:  90
Winning trades:  39
Losing Trades:  51
Benchmark Return:  2248.9531785306435
WIN rate:  43.333333333333336  %
Gross Profit:  3573.3856537157108
Net Profit:  3483.3856537157108
Average Profit:  38.70428504128567
MAX Drawdowns:  -25.876296889191885  %
Largest Win:  536.2987182514441
Average Win:  177.6964675330233
Largest Loss:  -226.71593602848884
Average Loss:  -65.81914862886664
Maximum Holding Time:  71
Average Holding Duration:  15.355555555555556
Maximum Dip:  22.671593602848883
Avg. Dip:  4.867685335043812
Sharpe Ratio:  0.25347078121347333
Sortino Ratio:  0.8733915339919821
returns:  3.9704285041285674  %


In [139]:
a

,trade number,entry,exit,holding duration,return,return %,drawdown
0,0,2018-01-02,2018-01-11,9,-62.997257,-6.299726,-0.185355
1,1,2018-01-12,2018-01-29,17,151.153874,15.115387,0.197255
2,2,2018-01-30,2018-02-11,12,174.756472,17.475647,0.125885
3,3,2018-02-12,2018-03-08,24,73.968019,7.396802,-0.202152
4,4,2018-03-09,2018-03-23,14,30.648864,3.064886,0.139377
...,...,...,...,...,...,...,...
85,85,2021-10-25,2021-11-09,15,-53.758826,-5.375883,0.115374
86,86,2021-11-10,2021-11-12,2,-57.829879,-5.782988,-0.057830
87,87,2021-11-13,2021-12-19,36,265.271727,26.527173,0.061349
88,88,2021-12-20,2022-01-04,15,-11.208038,-1.120804,-0.114928


In [140]:
daily_metrics=metrics(data,trailing_stop_loss(data,to_submit(data,output)))

Total Closed trades:  90
Winning trades:  38
Losing Trades:  52
Benchmark Return:  2248.9531785306435
WIN rate:  42.22222222222222  %
Gross Profit:  1387.263009097085
Net Profit:  1297.263009097085
Average Profit:  14.414033434412055
MAX Drawdowns:  -19.703510653063294  %
Largest Win:  340.42142306304413
Average Win:  108.96727254803238
Largest Loss:  -115.86012128743687
Average Loss:  -52.95179514861817
Maximum Holding Time:  29
Average Holding Duration:  7.288888888888889
Sharpe Ratio:  0.14584864444221382
Sortino Ratio:  0.5376622807918214
returns:  154.14033434412056  %


In [141]:
c=metrics(data,only_long_then_short(data,output))

Total Closed trades:  47
Winning trades:  19
Losing Trades:  28
Benchmark Return:  2248.9531785306435
WIN rate:  40.42553191489362  %
Gross Profit:  2851.0013184889476
Net Profit:  2804.0013184889476
Average Profit:  59.659602521041435
MAX Drawdowns:  -25.876296889191885  %
Largest Win:  735.1464286772573
Average Win:  243.2910804029299
Largest Loss:  -217.96301012440472
Average Loss:  -63.26890032738288
Maximum Holding Time:  72
Average Holding Duration:  16.340425531914892
Sharpe Ratio:  0.309859636602874
Sortino Ratio:  1.3163902782588712
returns:  606.5960252104144  %


In [142]:
c

,trade number,entry,exit,holding duration,return,return %,drawdown
0,0,2018-01-02,2018-01-11,9,-62.997257,-6.299726,-0.185355
1,1,2018-01-29,2018-01-30,1,-114.704957,-11.470496,-0.114705
2,2,2018-02-11,2018-03-08,25,122.013682,12.201368,-0.202152
3,3,2018-03-23,2018-03-27,4,-87.218642,-8.721864,-0.117317
4,4,2018-04-08,2018-05-08,30,316.716167,31.671617,-0.067929
5,5,2018-06-03,2018-06-11,8,-122.125554,-12.212555,-0.124315
6,6,2018-06-20,2018-06-23,3,-94.339623,-9.433962,-0.094340
7,7,2018-06-28,2018-06-29,1,-36.624386,-3.662439,-0.036624
8,8,2018-06-30,2018-08-01,32,194.091050,19.409105,-0.087883
9,9,2018-08-21,2018-09-07,17,-5.099225,-0.509922,-0.127877


In [143]:
d=metrics(data,trailing_stop_loss(data,only_long_then_short(data,output)))

Total Closed trades:  47
Winning trades:  19
Losing Trades:  28
Benchmark Return:  2248.9531785306435
WIN rate:  40.42553191489362  %
Gross Profit:  1549.2015262816944
Net Profit:  1502.2015262816944
Average Profit:  31.96173460173818
MAX Drawdowns:  -19.703510653063294  %
Largest Win:  513.9161528413531
Average Win:  151.3934360843991
Largest Loss:  -119.52852305287831
Average Loss:  -47.402634261496004
Maximum Holding Time:  21
Average Holding Duration:  8.340425531914894
Sharpe Ratio:  0.24903333835465224
Sortino Ratio:  1.0107930278757977
returns:  329.6173460173818  %
